# Get Links for Historical Rental Records for all Postcodes
## DO NOT RUN ALL, READ ALL CELLS FIRST
This notebook will get all links of rental records per postcode from: https://www.oldlistings.com.au/site-map?page=17&state=VIC

In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests import get
import re
import pickle
import time
from random import seed
from random import random
from random import randint

### no need to run this, but don't delete the following cell

In [ ]:
'''import requests
from bs4 import BeautifulSoup
import re
 
page = 18 
urls = []
for i in range(page):
    webpage = f'https://www.oldlistings.com.au/site-map?page={i}&state=VIC'
    reqs = requests.get(webpage)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    for link in soup.find_all('a'):
        href = link.get('href')
        if 'rent' in href:
            url = 'https://www.oldlistings.com.au'+href
            print(url)
            urls.append(url)'''

## get rental urls per postcode with regular expression
### website list saved (no need to run this, but don't delete the following cell)

In [96]:
'''

regex = r'https:/[/]www.oldlistings.com.au[/]\w+[-]\w+[/]VIC[/]\w+[+]?\w+[/]\d+[/]rent[/]\d?$'
rent_urls = []
for url in urls:
    rent_url = re.findall(regex, url)
    rent_urls.append(rent_url)
rent_urls
flat_list = [item for sublist in rent_urls for item in sublist]
flat_list


l = list(set(flat_list))
l.sort()
with open("HIS_url_links", "wb") as fp:
    pickle.dump(l, fp)
len(l)

'''

5388

In [73]:
import os
newpath = r'../../data/raw/historical_data' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

## get data function

In [19]:
def get_urls_daily(start, end):
    with open("HIS_url_links", "rb") as fp:
        result_urls = pickle.load(fp)
    result_urls_partial = result_urls[start:end]
    houses = []
    posts = []
    headers = {"User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Mobile Safari/537.36"}

    for i in range(len(result_urls_partial)):
        soup = BeautifulSoup(requests.get(result_urls_partial[i], headers=headers).text, 'html.parser')
        # in the html of the page, find all the bins with <li> and class:
        print(f'{i}.getting data from {result_urls_partial[i]}')
        house_data = soup.find_all("div", {"class":re.compile("property (odd|even) clearfix")})
        regex = r'(\d{4})'
        post = re.findall(regex, result_urls_partial[i])
        # random wait times
        value = random()
        scaled_value = 1 + (value * (9 - 5))
        # print(scaled_value)
        time.sleep(scaled_value)

        # no need this
        #for link in result_urls_partial:
        #    post = re.findall(regex, link)
        #    postcode = post[0][0:4]

        houses.extend(house_data)
        for c in range(len(house_data)):
            posts.append(post)
        
        
    count = 0
    data = pd.DataFrame()
    first = True

    count_p = 0
    for num in houses:
        # getting the price: make sure to test this code a few times by itself to understand exactly which parameters will work 
        current_priceTag = num.find_all('section', {"class":"grid-35 tablet-grid-35 price"})
        priceSection = num.find_all('section',{"class":"grid-100 historical-price"})
        priceTags = []
        rent_prices = []
        rent_dates = []

        try:
            current_price = re.search(r'\<h3\>(.*)\<\/h3\>', str(current_priceTag))
            current_date = re.search(r'\:[ ](.*)\<\/span\>', str(current_priceTag))
            rent_prices.append(current_price.group(1))
            rent_dates.append(current_date.group(1))
        except:
            print("Current rent value exception")

        priceTags = priceSection[0].find_all('li')
        for ps in priceTags:
            try:
                rent = re.search(r'\<\/span\>(.*)\<\/li\>', str(ps))
                rent_prices.append(rent.group(1))
                date = re.search(r'\<span\>(.*)\<\/span\>', str(ps))
                rent_dates.append(date.group(1))
            except:
                print("Historical values exception")

        try:
            lat = re.search(r'data\-lat\=\"(.*\d)\"[ ]', str(num)) 
            lng = re.search(r'data\-lng\=\"(.*\d)\"\>', str(num)) 
            latitude = lat.group(1)
            longitude = lng.group(1)
        except:
            print("Location values exception")

        try:
            nbed = num.find_all('p', {"class": "property-meta bed"})[0].text
            nbed = re.search(r'[A-Za-z][ ]:[ ](.*)', nbed).group(1)
        except IndexError:
            nbed = 'none'
        try:
            nbath = num.find_all('p', {"class": "property-meta bath"})[0].text
            nbath = re.search(r'[A-Za-z][ ]:[ ](.*)', nbath).group(1)
        except IndexError:
            nbath = 'none'
        try:
            ncar = num.find_all('p', {"class": "property-meta car"})[0].text
            ncar = re.search(r'[A-Za-z][ ]:[ ](.*)', ncar).group(1)
        except IndexError:
            ncar = 'none'
        try:
            type = num.find_all('p', {"class": "property-meta type"})[0].text
            type = re.search(r'[A-Za-z][ ]:[ ](.*)', type).group(1)
        except IndexError:
            type = 'none'
        try:
            address = num.find_all('h2', {"class":"address"})[0].text
        except IndexError:
            address = 'none'

        for z in zip(rent_prices, rent_dates):
            d = {"address":[address], "latitude":[latitude], "longitude":[longitude], 
                "nbed":[nbed], "nbath":[nbath], "ncar":[ncar], "historical_prices":[z[0]], 
                "type":[type],
                "historical_dates":[z[1]],
                "postcode":posts[count_p]}

            if first:
                first = False
                data = pd.DataFrame.from_dict(d)
            else:
                data = pd.concat([data, pd.DataFrame.from_dict(d)])

        count_p += 1
    print(data.head(10))
    data.to_csv(f'../../data/raw/historical_data/{start}_{end}_historical.csv')

## Exaple demonstration: calling function 
### 100 websites will be scraped per call, will retrieve round 10k data each call
### no need to run the following cells (can view the output)

In [ ]:
# example: call the first 100 urls.
# get_urls_daily(0,5000)


In [ ]:
# example: call the 100-200 urls. (if you wish to run more than 200, CHANGE to another wifi and call)
# get_urls_daily(100, 200)

## What the team needs to do (Write and Run from here)
### Call the function at least twice a day (just to be safe, only make 2 calls with 1 wifi)
following is an example (do not call yet)

**07/09/2022 Wed**
*   grace: 0-200

*   jin: 200-400

*   john: 400-600

*   katherine: 600-800

*   philip: 800-1000


In [ ]:
# grace
get_urls_daily(0, 100) # first call 
get_urls_daily(100, 200) # second call 

In [ ]:
# jin
get_urls_daily(100, 200)
get_urls_daily(200, 300)

In [ ]:
# etc.

comment out the function after calling

**08/09/2022 Thur** (this schedule may be changed)
*   grace: 1000-1200

*   jin: 1200-1400

*   john: 1400-1600

*   katherine: 1600-1800

*   philip: 1800-2000


In [ ]:
# grace
get_urls_daily(1000, 1100)
get_urls_daily(1100, 1200)

In [ ]:
# etc.